Na stranici http://steinlib.zib.de/steinlib.php nalazi se biblioteka podataka vezana za Štajnerove grafove.  Test podaci na ovoj stranice sadrže informacije o pojedinačnim Štajnerovim grafovima. Oni su  čuvani u .stp formatu koji se koristi samo za čuvanje podataka o Štajnerovim grafovima. Postoji python modul za parsiranje ovog formata koji je opisan na https://github.com/leandron/steinlib.

In [1]:
!pip install steinlib

In [2]:
from steinlib.instance import SteinlibInstance
from steinlib.parser import SteinlibParser
from steinlib.section import SectionParser

import sys

class MySteinLibGraph(SteinlibInstance):

    def comment(self, raw_args, list_args):
        print("Comment section found")

    def comment__end(self, raw_args, list_args):
        print("Comment section end")

    def coordinates(self, raw_args, list_args):
        print("Coordinates section found")

    def eof(self, raw_args, list_args):
        print("End of file found")

    def graph(self, raw_args, list_args):
        print("Graph section found")
        print(raw_args, list_args)

    def header(self, raw_args, list_args):
        print("Header found")

    def terminals(self, raw_args, list_args):
        print("Terminals section found")
        

Nisam se snašao da koristim steinlib python modul. Pa u nastavku pišem kod koji će služiti da se učita graf iz datoteka formata .stp.

In [3]:
def loadGraphFromFile(file_name):

        n_nodes = 0
        n_edges = 0
        n_terminals = 0

        edges = []
        terminals = []

        with open(file_name, 'r') as file:

            for line in file:
                word_list = line.split()

                if(len(word_list) == 0):
                    continue

                if(word_list[0] == 'Nodes'):
                    n_nodes = int(word_list[1])

                if(word_list[0] == 'Edges'):
                    n_edges = int(word_list[1])

                if(word_list[0] == 'Terminals'):
                    n_terminals = int(word_list[1])

                if(word_list[0] == 'E'):
                    edges.append([int(word_list[1]), int(word_list[2]), int(word_list[3])])

                if(word_list[0] == 'T'):
                    terminals.append(int(word_list[1]))
                    
        return [n_nodes, n_edges, n_terminals, edges, terminals]

Napravljena je klasa za čuvanje informacija o učitanim grafovima.

In [4]:
class SteinerTree:
    
    def __init__(self, file_name, opt):
        
        initList = loadGraphFromFile(file_name)
        
        self.name = file_name[-7:-4]
        self.opt = opt
        self.numNodes = initList[0]
        self.numEdges = initList[1]
        self.numTerms = initList[2]
        self.edges = initList[3]
        self.terminals = initList[4]

Provera da li radi učitavanje grafova.

In [5]:
st = SteinerTree('b01.stp', 82)
print(st.name)
print(st.opt)
print(st.numNodes)
print(st.numEdges)
print(st.numTerms)
print(st.edges)
print(st.terminals)

b01
82
50
63
9
[[2, 8, 8], [2, 21, 7], [2, 32, 2], [4, 5, 8], [7, 29, 7], [11, 3, 7], [14, 31, 9], [17, 6, 7], [17, 42, 6], [18, 19, 2], [18, 28, 1], [18, 43, 1], [19, 2, 5], [20, 7, 3], [20, 14, 7], [20, 16, 8], [20, 27, 2], [20, 38, 8], [20, 40, 10], [20, 48, 2], [21, 12, 7], [21, 17, 5], [21, 18, 10], [22, 10, 6], [22, 20, 2], [22, 21, 2], [22, 40, 8], [22, 43, 7], [25, 34, 4], [27, 34, 4], [28, 5, 8], [28, 24, 5], [29, 9, 5], [29, 33, 7], [30, 5, 4], [30, 15, 1], [30, 16, 2], [33, 35, 3], [34, 20, 10], [34, 30, 2], [36, 2, 8], [36, 4, 6], [36, 11, 9], [36, 39, 7], [36, 49, 9], [36, 50, 10], [40, 15, 10], [40, 23, 3], [41, 1, 5], [41, 22, 8], [41, 25, 5], [41, 36, 2], [41, 44, 7], [41, 47, 7], [42, 6, 9], [42, 46, 10], [44, 24, 8], [44, 39, 3], [45, 26, 6], [45, 28, 1], [47, 37, 3], [47, 45, 10], [50, 13, 1]]
[48, 49, 22, 35, 27, 12, 37, 34, 24]


Učitavanje informacija o grafovima iz skupa podataka B.

In [6]:
bGraphList = [
    ('b01.stp', 82),
    ('b02.stp', 83),
    ('b03.stp', 138),
    ('b04.stp', 59),
    ('b05.stp', 61),
    ('b06.stp', 122),
    ('b07.stp', 111),
    ('b08.stp', 104),
    ('b09.stp', 220),
    ('b10.stp', 86),
    ('b11.stp', 88),
    ('b12.stp', 174),
    ('b13.stp', 165),
    ('b14.stp', 235),
    ('b15.stp', 318),
    ('b16.stp', 127),
    ('b17.stp', 131),
    ('b18.stp', 218)
]

In [7]:
import os

bDataPath = os.path.abspath(os.getcwd()) + '\\..\\data\\B'

print(bDataPath)
filenames = os.listdir(bDataPath)
print(filenames)

bGraphs = []

for st_args in bGraphList:
    graphpath = bDataPath + '\\' + st_args[0]
    graphopt = st_args[1]
    bGraphs.append(SteinerTree(graphpath, graphopt))
    
print(len(bGraphs))
print(bGraphs[9].name)

G:\RI\Projekat\RI-Min-Steiner-Tree\code\..\data\B
['b01.stp', 'b02.stp', 'b03.stp', 'b04.stp', 'b05.stp', 'b06.stp', 'b07.stp', 'b08.stp', 'b09.stp', 'b10.stp', 'b11.stp', 'b12.stp', 'b13.stp', 'b14.stp', 'b15.stp', 'b16.stp', 'b17.stp', 'b18.stp']
18
b10


In [128]:
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree

In [142]:
import numpy as np

#nasumicno generise populaciju cija jedinka predstavlja skup izabranih cvorova koji ulaze Steinerovo drvo
#konacno resenje je minimalno razapinjuce stablo koje sadrzi sve ove cvorove
def generatePopulation(population_size, solution_len):
    
    pop = []
    
    for i in range(population_size):
        
        sol = []
        
        for j in range(solution_len):
            
            sol.append(round(np.random.rand()))
            
        pop.append(sol)
            
    return np.array(pop)

#evaluira se svaki clan populacije, racunanjem vrednosti minimalnog razapinjuceg stabla
def evaluatePopulation(pop, population_size, solution_len, terminal_list, adj_matrix):
    
    N = len(adj_matrix)
    
    selected_nodes = terminal_list
    
    population_eval_array = []
    
    #izdvajaju se svi izabrani cvorovi (terminali + cvorovi iz jedinke populacije)
    for p in range(population_size):
        
        i = 0
        
        for t in range(len(terminal_list)):
            
            if(terminal_list[t] == 0):
                
                i = i+1
                
                if(pop[p][i-1] == 1):
                    
                    selected_nodes[t] = 1
        
        #pravi se nova matrica susedstva za izabrane cvorove
        M = sum(selected_nodes)
        
        new_adj_matrix = np.zeros((int(M), int(M)))
        
        m = 0
        
        for i in range(N):
            
            if(selected_nodes[i] == 1):
                
                n = 0
                
                for j in range(N):
                    
                    if(selected_nodes[j] == 1):
                        
                        new_adj_matrix[m][n] = adj_matrix[i][j]
                        
                        n = n+1
                m = m+1
        
        
        X = csr_matrix(new_adj_matrix.tolist())
        #vraca gornje-trougaonu matricu povezanosti minimalnog razapinjuceg stabla
        Tcsr = minimum_spanning_tree(X)
        population_eval_array.append(sum(sum(Tcsr.toarray().astype(int))))
        
    return population_eval_array
        

def geneticAlgorithm(st: SteinerTree):
    
    N = st.numNodes
    edge_number = st.numEdges
    term_number = st.numTerms
    optimal = st.opt
    edges = st.edges
    terminals = st.terminals
    
    terminal_list = np.zeros(N)
    
    for t in terminals:
        terminal_list[t] = 1
    
    #success flag
    success = 0
    solution = optimal + 1
    
    adj_matrix = np.zeros((N, N))
    
    for e in edges:
        adj_matrix[e[0]-1][e[1]-1] = e[2]
        adj_matrix[e[1]-1][e[0]-1] = e[2]
    
    total_value = np.sum(adj_matrix) / 2
    
    
    population_size = 10
    solution_len = N - term_number
    
    pop = generatePopulation(population_size, solution_len)
    
    pop_eval = evaluatePopulation(pop, population_size, solution_len, terminal_list, adj_matrix)
    
    print(pop_eval)

In [143]:
geneticAlgorithm(bGraphs[0])

[94, 196, 241, 230, 228, 228, 228, 228, 238, 238]
